In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('full_data.csv')
df.head()

題目一：基礎分析
情境： 你剛剛加入一家公司，負責分析他們的客戶購買行為以改進營銷策略。

Question： 請列出目前客戶資料中最受歡迎的五個產品是哪些？這些產品的銷售量和總銷售金額各是多少？

提示： 使用客戶資料和產品資料進行合併，計算每個產品的銷售量和總銷售金額。

In [ ]:
top5_product= df.groupby('ProductName')[['Quantity','TotalPrice']].sum().head(5).reset_index()
top5_product['rank'] = top5_product['Quantity'].rank(ascending=False).astype(int)
top5_product= top5_product.sort_values('rank')
top5_product.set_index('rank',inplace=True)
print(top5_product)

題目二：客戶分群分析
情境： 公司想要了解客戶的消費習慣，以便設計針對性的行銷活動。

Question： 請對客戶進行分群，基於他們的購買頻率和總購買金額，分析出至少三個不同的客戶群體。每個群體中客戶的特徵和購買行為有何不同？

提示： 使用客戶資料計算每個客戶的購買頻率和總購買金額，再根據這些指標進行客戶分群。

In [ ]:
df.info()

In [ ]:
daily_spend = df.groupby(['CustomerID', 'PurchaseDate'])['TotalPrice'].sum().reset_index()
print(daily_spend)

In [ ]:
unique_purchase_df = df.drop_duplicates(subset=['CustomerID', 'PurchaseDate'])
unique_purchase_df = unique_purchase_df [['CustomerID', 'PurchaseDate']]
# print(unique_purchase_df[['CustomerID','PurchaseDate']])
daily_spend = pd.merge(unique_purchase_df,daily_spend)
daily_spend = daily_spend[['CustomerID','PurchaseDate','TotalPrice']]
print(daily_spend)

In [ ]:
daily_spend['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])
daily_spend.head()

In [ ]:
average_spend = daily_spend.groupby('CustomerID')['TotalPrice'].mean().reset_index()
print(average_spend)

In [ ]:
year_spend = daily_spend.copy()
year_spend['year'] = year_spend['PurchaseDate'].dt.year
year_analysis = year_spend.groupby(['CustomerID', 'year']).agg({
    'PurchaseDate': 'count',  # 計算次數
    'TotalPrice': 'sum'       # 計算該年總金額
}).reset_index()
# year_spend['month'] = year_spend['PurchaseDate'].dt.month
# year_spend['month'] = year_spend['PurchaseDate'].dt.month

In [ ]:
year_analysis.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 先取得前 10 位客戶的 ID 清單
top_10_customers = year_analysis['CustomerID'].unique()[:10]

# 2. 建立一個大圖表，設定為 5 列 2 欄 (共 10格)
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 20))
fig.subplots_adjust(hspace=0.4, wspace=0.3) # 調整子圖之間的間距
axes = axes.flatten() # 將 2D 矩陣轉為 1D 清單，方便迴圈操作

# 3. 用迴圈幫每一位客戶畫圖
for i, customer_id in enumerate(top_10_customers):
    # 篩選該位客戶的資料
    customer_data = year_analysis[year_analysis['CustomerID'] == customer_id]
    
    # 排序年份確保線條順序正確
    customer_data = customer_data.sort_values('year')
    
    # 在對應的格子 (axes[i]) 畫上折線圖
    sns.lineplot(ax=axes[i], data=customer_data, x='year', y='PurchaseDate', marker='o', color='b')
    
    # 設定每張小圖的標題 (只顯示 ID 的前 8 位方便辨識)
    axes[i].set_title(f'Customer: {customer_id[:8]}...', fontsize=12)
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('PurchaseDate')
    axes[i].set_xticks(customer_data['year'].unique()) # 強制顯示所有年份

# 4. 加入整張大表的總標題
plt.suptitle('Annual Spending Trends for Top 10 Customers', fontsize=20, y=1.02)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 先取得前 10 位客戶的 ID 清單
top_10_customers = year_analysis['CustomerID'].unique()[:10]

# 2. 建立一個大圖表，設定為 5 列 2 欄 (共 10格)
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 20))
fig.subplots_adjust(hspace=0.4, wspace=0.3) # 調整子圖之間的間距
axes = axes.flatten() # 將 2D 矩陣轉為 1D 清單，方便迴圈操作

# 3. 用迴圈幫每一位客戶畫圖
for i, customer_id in enumerate(top_10_customers):
    # 篩選該位客戶的資料
    customer_data = year_analysis[year_analysis['CustomerID'] == customer_id]
    
    # 排序年份確保線條順序正確
    customer_data = customer_data.sort_values('year')
    
    # 在對應的格子 (axes[i]) 畫上折線圖
    sns.lineplot(ax=axes[i], data=customer_data, x='year', y='TotalPrice', marker='o', color='b')
    
    # 設定每張小圖的標題 (只顯示 ID 的前 8 位方便辨識)
    axes[i].set_title(f'Customer: {customer_id[:8]}...', fontsize=12)
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Total Spend')
    axes[i].set_xticks(customer_data['year'].unique()) # 強制顯示所有年份

# 4. 加入整張大表的總標題
plt.suptitle('Annual Spending Trends for Top 10 Customers', fontsize=20, y=1.02)
plt.show()

In [ ]:
#購買頻率
customerda= df.groupby('CustomerID')[['PurchaseAmount', 'TotalPrice']].sum().reset_index()
frequen = df.groupby('CustomerID')['PurchaseDate'].nunique().reset_index()
customerda = pd.merge(customerda,frequen)
customerda.describe()


In [ ]:
customerda

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(customerda['PurchaseDate'], customerda['TotalPrice'])
plt.xlabel('Frequency (PurchaseDate Count)')
plt.ylabel('Monetary (TotalPrice Sum)')
plt.title('Customer Distribution')
plt.show()

In [ ]:
from sklearn.cluster import KMeans 

km = KMeans(3)
clusters = km.fit_predict(customerda[['PurchaseDate', 'TotalPrice']])
clusters

2 features: array([1, 0, 2, 2, 1, 0, 0, 2, 0, 1], dtype=int32)


In [ ]:
customerda['Clusters'] = pd.Series(clusters)
customerda


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(customerda['PurchaseDate'], customerda['TotalPrice'],c=customerda['Clusters'])
plt.xlabel('Frequency (PurchaseDate Count)')
plt.ylabel('Monetary (TotalPrice Sum)')
plt.title('Customer Distribution')
plt.show()

題目三：產品分析
情境： 公司希望了解不同類型產品的銷售情況，以評估市場需求。

Question： 請分析每個產品類別的銷售量和銷售金額占比。哪些產品類別是銷售的主要貢獻者？是否有某些產品類別的銷售表現較差，需要進行改進？

提示： 使用產品資料進行分析，計算每個產品類別的銷售量和銷售金額，再計算其占比。

In [ ]:
df.head()

In [ ]:
sale_product= df.copy()
sale_analysis = sale_product.groupby('ProductName').agg({
    'TotalPrice': 'sum',  # 計算次數
    'Quantity': 'sum',
    'Price': 'first' ,# 計算該年總金額
}).reset_index()

In [ ]:
sale_analysis = sale_analysis.sort_values('Quantity',ascending=False)
sale_analysis['rank'] = sale_analysis['Quantity'].rank(ascending=False).astype(int)
sale_analysis.set_index('rank',inplace=True)
print(sale_analysis)

In [ ]:
total_revenue = sale_analysis['TotalPrice'].sum() 
total_Quantity = sale_analysis['Quantity'].sum() 
sale_analysis['TotalPrice%'] = ( sale_analysis['TotalPrice'] / total_revenue )*100
sale_analysis['Quantity%'] = ( sale_analysis['Quantity'] / total_Quantity )*100
print(sale_analysis)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.pie(sale_analysis['TotalPrice%'], labels=sale_analysis['ProductName'], autopct='%1.1f%%')
plt.title('Sales Revenue Share by Category')
plt.show()

題目四：客戶忠誠度分析
情境： 公司希望了解客戶的忠誠度和購買趨勢，以保持現有客戶和吸引新客戶。

Question： 請分析客戶的購買頻率和購買金額的分佈情況。哪些客戶是公司的忠實顧客？有沒有具有潛力的新客戶？

提示： 使用客戶資料計算每個客戶的購買頻率和購買金額，進行客戶忠誠度的分析。

In [ ]:
maxdate=df['PurchaseDate'].max()
mindate=df['PurchaseDate'].min()
maxdate = pd.to_datetime(maxdate)
mindate = pd.to_datetime(mindate)
total_date=(maxdate-mindate).days+1


print(total_date)

In [ ]:
customer_data_js= pd.read_json('customer_data.json')
customer_data_js

In [ ]:
customerda = pd.merge(customer_data_js,customerda)
customerda['avg_days'] = (customerda['PurchaseDate']/total_date)*100
customerda['avg_price'] =  customerda['TotalPrice']/customerda['PurchaseDate']
customerda_any = customerda[['ScientistName','PurchaseAmount','TotalPrice','PurchaseDate','avg_price','Clusters']]

In [ ]:
customerda_any

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 設定畫布
plt.figure(figsize=(10, 6))

# X軸為頻率，Y軸為總金額，點的大小代表平均單價
sns.scatterplot(data=customerda_any, x='PurchaseDate', y='TotalPrice', size='avg_price', alpha=0.7)

# 加上平均線作為輔助判斷
plt.axvline(customerda_any['PurchaseDate'].mean(), color='red', linestyle='--', label='Average Frequency')
plt.axhline(customerda_any['TotalPrice'].mean(), color='blue', linestyle='--', label='Average Spend')

plt.title('Customer Loyalty Analysis: Frequency vs. Total Spend')
plt.legend()
plt.show()

In [ ]:
# 1. 計算全體的平均頻率與平均總金額
avg_freq = customerda_any['PurchaseDate'].mean()
avg_spend = customerda_any['TotalPrice'].mean()

# 2. 定義分類邏輯
def classify_loyalty(row):
    if row['PurchaseDate'] >= avg_freq and row['TotalPrice'] >= avg_spend:
        return '核心忠實客戶'
    elif row['PurchaseDate'] < avg_freq and row['avg_price'] >= customerda_any['avg_price'].mean():
        return '高單價潛力新客'
    elif row['PurchaseDate'] >= avg_freq and row['TotalPrice'] < avg_spend:
        return '忠實小額客群'
    else:
        return '一般/需注意客戶'

# 3. 套用分類
customerda_any['Status'] = customerda_any.apply(classify_loyalty, axis=1)

# 4. 查看各分類的人數分佈
status_counts = customerda_any['Status'].value_counts()
print(status_counts)

In [ ]:
customerda_any

In [ ]:
# 看看各標籤客戶貢獻的總金額百分比
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
loyalty_revenue = customerda_any.groupby('Status')['TotalPrice'].sum()
loyalty_revenue.plot(kind='pie', autopct='%1.1f%%', title='Revenue Contribution by Loyalty Status')

題目五：預測分析
情境： 公司希望能夠預測未來一段時間內的銷售趨勢，以制定適當的庫存和營銷策略。

Question： 基於目前的客戶購買行為，請預測下個月每個產品的銷售量和銷售金額。你會如何進行這樣的預測分析？

提示： 使用時間序列或機器學習模型來預測下個月的銷售量和銷售金額，可以考慮客戶的購買趨勢和產品的季節性影響。

In [ ]:

df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])
df['YearMonth'] = df['PurchaseDate'].dt.to_period('M')

In [ ]:
df.head()

In [ ]:
sale_pre = df[['ProductName','Quantity','Price','TotalPrice','YearMonth']]

In [ ]:
sale_pre_new = sale_pre.groupby(['ProductName','YearMonth']).agg({
    'TotalPrice': 'sum',  # 計算次數
    'Quantity': 'sum',
    'Price': 'first' ,# 計算該年總金額
}).reset_index()

In [ ]:
# 簡單移動平均法 (Moving Average)： 取過去 3 個月的平均值作為下個月預測（適合穩定銷售的產品）。
maxdate=df['PurchaseDate'].max()
print(maxdate) 



In [ ]:
sale_pre_new

In [ ]:
target_months = [pd.Period('2024-03', freq='M'), 
                 pd.Period('2024-04', freq='M'), 
                 pd.Period('2024-05', freq='M')]

mark= sale_pre_new['YearMonth'].isin(target_months)
filtered_df = sale_pre_new[mark]
print(filtered_df)

In [ ]:
filtered_df = filtered_df.groupby(['ProductName','YearMonth']).agg({
'TotalPrice':'sum',
'Quantity':'sum',
'Price':'first'
}).reindex()

In [ ]:
filtered_df

In [ ]:
#transform 會把算出來的平均值，廣播（Broadcast）到該產品對應的 3-5 月每一行。這在做特徵工程或對比分析時非常有用。
filtered_df['Avg_Quantity_3M'] = filtered_df.groupby('ProductName')['Quantity'].transform('mean')
filtered_df['Avg_TotalPrice_3M'] = filtered_df.groupby('ProductName')['TotalPrice'].transform('mean')

In [ ]:
result_df = filtered_df.reset_index()

In [ ]:
prediction_next_month = result_df.groupby('ProductName').agg({
    'Avg_Quantity_3M': 'first',
    'Avg_TotalPrice_3M': 'first'
}).rename(columns={
    'Avg_Quantity_3M': 'Predicted_Quantity_June',
    'Avg_Price_3M': 'Predicted_Sales_June'
})

In [ ]:
prediction_next_month 

In [ ]:
filtered_df.reset_index()
print(filtered_df)

In [ ]:
sale_pre_new

In [ ]:
sale_pre_new = pd.get_dummies(sale_pre_new,columns=['ProductName']
)


In [ ]:
sale_pre_new

In [ ]:
sale_pre_new['Month_Code'], _ = pd.factorize(sale_pre_new['YearMonth'])

In [ ]:
sale_pre_new.head()

In [ ]:
sale_pre_new['Quantity']=sale_pre_new.pop('Quantity')

In [ ]:
sale_pre_new.info()

In [ ]:
X = sale_pre_new.drop(columns=['Quantity','TotalPrice','YearMonth'])
y = sale_pre_new['Quantity']

In [ ]:
X=X.values
y=y.values.reshape(-1,1)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split #切割方法
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.linear_model import LinearRegression
regressor_linear = LinearRegression() #線性回歸 modul
regressor_linear.fit(X_train, y_train) #把資料丟進去

In [ ]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from collections import Counter

# from sklearn.datasets import load_boston
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import r2_score
import numpy as np

# R2 score，即決定係數，反映Y的全部變異能通過迴歸關係被所有X解釋的比例。
y_pred_linear_train = regressor_linear.predict(X_train)
r2_score_linear_train = r2_score(y_train, y_pred_linear_train)

y_pred_linear_test = regressor_linear.predict(X_test)
r2_score_linear_test = r2_score(y_test, y_pred_linear_test)

# RMSE一般指均方根誤差(標準誤差)，可衡量預測值和實際值之間的平均差異，藉此估計預測模型預測目標值的準確度
rmse_train = (np.sqrt(mean_squared_error(y_train, y_pred_linear_train)))
rmse_test = (np.sqrt(mean_squared_error(y_test, y_pred_linear_test)))

print('R2_score (train): ', r2_score_linear_train)
print('R2_score (test): ', r2_score_linear_test)
print("RMSE: ", rmse_train) #越低越好 標準誤差
print("RMSE: ", rmse_test)